In [1]:
import streamlit as st 
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain_community.document_loaders import SpiderLoader, PyPDFLoader, WebBaseLoader, DirectoryLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from dotenv import load_dotenv
import os
import shutil

In [ ]:
#note: run using python -m streamlit run app.py instead 

load_dotenv()
FAISS_PATH = "/Users/macbook/Desktop/code/proj/jochatbot/faiss_data"
CHROMA_PATH = "/Users/macbook/Desktop/code/proj/jochatbot/chroma_data"
DIR_PATH_TXT = "/Users/macbook/Desktop/code/proj/jochatbot/data/txt"
DIR_PATH_CSV = '/Users/macbook/Desktop/code/proj/jochatbot/data/csv'

os.environ['OPENAI_API_KEY'] = os.environ.get('API_KEY_PERSONAL')

In [3]:
loader_txt = DirectoryLoader(DIR_PATH_TXT, glob="*.txt")
# loader_csv = DirectoryLoader(DIR_PATH_CSV, glob="*.csv")
loader_pdf = PyPDFLoader("data/pdf/1. คู่มือ FXdreema (Jobot Invest).pdf", extract_images=True)

In [4]:

# loader_meta = SpiderLoader("https://www.mql5.com/en/code/mt5",
#                            api_key="sk-c5a3c4fd-2a64-4e06-9780-56e78faf5625",
#                            mode="crawl")
# loader_forex = SpiderLoader("https://www.forexstrategiesresources.com/",
#                             api_key="sk-c5a3c4fd-2a64-4e06-9780-56e78faf5625",
#                             mode="crawl")

loader_all = MergedDataLoader(loaders=[loader_txt, loader_pdf])
documents = loader_all.load()
docs_processed = filter_complex_metadata(documents)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)
chunks = text_splitter.split_documents(documents)


In [ ]:
db = FAISS.from_documents(chunks, OpenAIEmbeddings())
db.save_local(FAISS_PATH)

In [ ]:
db = Chroma.from_documents(chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH)
db.persist()